In [11]:
%pip uninstall calvin-utils

Found existing installation: calvin-utils 0.1
Uninstalling calvin-utils-0.1:
  Would remove:
    /usr/local/Caskroom/mambaforge/base/envs/nimlab_py310/lib/python3.10/site-packages/calvin-utils.egg-link
Proceed (Y/n)? ^C
ERROR: Operation cancelled by user
Note: you may need to restart the kernel to use updated packages.


In [7]:
import sys
print(sys.executable)


/usr/local/Caskroom/mambaforge/base/envs/nimlab_py310/bin/python


In [8]:
import sys
print(sys.path)


['/Users/cu135/Library/CloudStorage/OneDrive-Personal/OneDrive_Documents/Work/Software/Research/nimlab/notebooks/neuroimaging_notebooks/server_submission_notebooks', '/usr/local/Caskroom/mambaforge/base/envs/nimlab_py310/lib/python310.zip', '/usr/local/Caskroom/mambaforge/base/envs/nimlab_py310/lib/python3.10', '/usr/local/Caskroom/mambaforge/base/envs/nimlab_py310/lib/python3.10/lib-dynload', '', '/Users/cu135/.local/lib/python3.10/site-packages', '/usr/local/Caskroom/mambaforge/base/envs/nimlab_py310/lib/python3.10/site-packages', '/Users/cu135/Library/CloudStorage/OneDrive-Personal/OneDrive_Documents/Work/Software/Research/nimlab/calvin_utils']


In [2]:
#Imports
%pip install -e /Users/cu135/Library/CloudStorage/OneDrive-Personal/OneDrive_Documents/Work/Software/Research/nimlab/calvin_utils
import sys
import os
import glob as glob
import pandas as pd
import numpy as np
np.set_printoptions(precision=3, suppress=True)

Obtaining file:///Users/cu135/Library/CloudStorage/OneDrive-Personal/OneDrive_Documents/Work/Software/Research/nimlab/calvin_utils
  Preparing metadata (setup.py) ... done
  Attempting uninstall: calvin-utils
    Found existing installation: calvin-utils 0.1
    Uninstalling calvin-utils-0.1:
      Successfully uninstalled calvin-utils-0.1
  Running setup.py develop for calvin-utils
Note: you may need to restart the kernel to use updated packages.


In [3]:
from calvin_utils.file_utils.dataframe_utilities import preprocess_colnames_for_regression


ModuleNotFoundError: No module named 'calvin_utils'

In [3]:
analysis = 'response_topology/voxelwise_mediated_moderation/age_mediated_by_grey_matter'
clin_path = r'/Users/cu135/Dropbox (Partners HealthCare)/resources/datasets/BIDS_AD_DBS_FORNIX/study_metadata/derivative_metadata/quantitative_atrophy/grey_matter_damage_score_and_outcomes/grey_matter_damage_score_and_outcomes.csv'
out_dir = os.path.join(os.path.dirname(clin_path), f'{analysis}')
save = True
if os.path.exists(out_dir):
    pass
else:
    os.makedirs(out_dir)

Define Data to Submit

In [ ]:
#----------------------------------------------------------------user input above----------------------------------------------------------------
data_df = pd.read_csv(clin_path)
    
# #Remove subjects
# outlier_index=[11, 47, 48, 49]
# data_df = data_df.drop(index=outlier_index)
# data_df.reset_index(drop=True, inplace=True)

data_df

Outocme Data

In [ ]:
subject_column = 'Patient # CDR, ADAS'
outcome_column = '% Change from baseline (ADAS-Cog11)'
#----------------------------------------------------------------DO NOT MODIFY--------------------------------------------------------
outcomes_df = pd.DataFrame()
outcomes_df['outcome'] = data_df.loc[:, [outcome_column]]

#Standardize the data
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler() 
outcomes_df.iloc[:,:] = scaler.fit_transform(outcomes_df.iloc[:,:])


outcomes_df['subject_id'] = data_df.loc[:, [subject_column]]
# outcomes_df['subject_id'] = [id.split('_')[0] for id in data_df[subject_column].to_list()] #<------------------------problem code. must generalize to extract subject id better
outcomes_df.set_index('subject_id', inplace=True)

# data_df = data_df.set_index('Patient # CDR, ADAS')
# data_df['subject_id'] = data_df.index
# data_df['outcome'] = data_df

# Convert the 'subject_id' column to strings for each DataFrame
outcomes_df.index = outcomes_df.index.astype(str)
display(outcomes_df)

Clinical Covariates

In [2]:
subject_column = 'Patient # CDR, ADAS'
clinical_information_column = 'Age'
#----------------------------------------------------------------DO NOT MODIFY--------------------------------------------------------
clinical_df_1 = pd.DataFrame()
clinical_df_1['clinical_data'] = data_df.loc[:, [clinical_information_column]]

clinical_df_1['subject_id'] = data_df.loc[:, [subject_column]]
# clinical_df_1['subject_id'] = [id.split('_')[0] for id in data_df[subject_column].to_list()] 
clinical_df_1.set_index('subject_id', inplace=True)

# Convert the 'subject_id' column to strings for each DataFrame
clinical_df_1.index = clinical_df_1.index.astype(str)
display(clinical_df_1)

NameError: name 'data_df' is not defined

Voxelwise Data

In [ ]:
from calvin_utils.file_utils.import_matrices import import_matrices_from_folder
from calvin_utils.nifti_utils.generate_nifti import nifti_from_matrix
from nimlab import datasets as nimds
import numpy as np
from nilearn import image, plotting, maskers

#get conectivity values of interest
matrix_path = '/Users/cu135/Dropbox (Partners HealthCare)/resources/datasets/BIDS_AD_DBS_FORNIX/connectivity_data/vta_published_t_connectivity'
matrix = import_matrices_from_folder(matrix_path, file_pattern='/*_T*.nii*')

#Standardize matrix via z score
prepped_matrix = pd.DataFrame()
for column in matrix.columns:
    print(column)
    new_name = int(column.split('_')[0]) #.split('T')[1]) #<----------------------------------THIS IS A BUG-CREATOR. MUST BE TAILORED TO PATIENT DATA

#Set patients to those in the clinical data dataframe
prepped_matrix = prepped_matrix.transpose()
prepped_matrix['subject_id'] = [str(col).split('_')[0] for col in prepped_matrix.index]
prepped_matrix.set_index('subject_id', inplace=True)
neuroimaging_df_1 = prepped_matrix
neuroimaging_df_1.index = neuroimaging_df_1.index.astype(str)
#Display results
display(neuroimaging_df_1)

Voxelwise Data 2

In [ ]:
#Import Another Dataframe if desired
from calvin_utils.file_utils.import_matrices import import_matrices_from_folder
from calvin_utils.nifti_utils.generate_nifti import nifti_from_matrix
from nimlab import datasets as nimds
import numpy as np
from nilearn import image, plotting, maskers

#get conectivity values of interest
matrix_path = '/Users/cu135/Dropbox (Partners HealthCare)/resources/datasets/BIDS_AD_DBS_FORNIX/cat12/cat12_ultrafine-reg/CAT12.8.2_2170'
neuroimaging_df_2 = import_matrices_from_folder(matrix_path, file_pattern='/*/*/*/*mwp1*resamp*.nii*')
performed_z_score = False 
#----------------------------------------------------------------DO NOT MODIFY!----------------------------------------------------------------

#Set patients to those in the clinical data dataframe
# prepped_matrix = prepped_matrix.loc[:, data_df.index]

neuroimaging_df_2 = neuroimaging_df_2.transpose()
neuroimaging_df_2['subject_id'] = [col.split('_')[0] for col in neuroimaging_df_2.index]
neuroimaging_df_2.set_index('subject_id', inplace=True)

neuroimaging_df_2.index = neuroimaging_df_2.index.astype(str)


#Display results
display(neuroimaging_df_2)

Masking

In [ ]:
#Mask Dataframes
mask_path = '/Users/cu135/Dropbox (Partners HealthCare)/resources/datasets/BIDS_AD_DBS_FORNIX/response_topology/voxelwise_glm/stim_by_age/palm_results/palm_statistic_topology/generated_nifti.nii' #None #
masking_df = None # # 
#----------------------------------------------------------------DO NOT MODIFY
from calvin_utils.nifti_utils.matrix_utilities import mask_matrix
# def mask_matrix(df_1, mask_path=None, mask_threshold=0.2, mask_by='rows', dataframe_to_mask_by=None):

neuroimaging_df_1 = mask_matrix(neuroimaging_df_1, mask_path=mask_path, mask_threshold=0, mask_by='columns', dataframe_to_mask_by=masking_df)
neuroimaging_df_2 = mask_matrix(neuroimaging_df_2, mask_path=mask_path, mask_threshold=0, mask_by='columns', dataframe_to_mask_by=masking_df)

Optional - Additional Preprocessing

In [ ]:
neuroimaging_df_2 = neuroimaging_df_2.reindex(neuroimaging_df_1.index).dropna()
neuroimaging_df_2

# Save Dataframes for Access by Script
    """
    Saves DataFrames to CSV files and returns the paths.

    Parameters:
    - outcome_dfs (list): A list of outcome DataFrames.
    - covariate_dfs (list): A list of covariate DataFrames.
    - voxelwise_dfs (list): A list of voxelwise DataFrames.
    - path_to_dataframes (str): The directory where the DataFrames will be saved.

    Returns:
    - dict: A dictionary containing lists of paths for each DataFrame type.
    """

In [ ]:
from calvin_utils.file_utils.dataframe_utilities import save_dataframes_to_csv

df_paths_dict = save_dataframes_to_csv(outcome_dfs = [outcomes_df], 
                                       covariate_dfs = [clinical_df_1],
                                       voxelwise_dfs = [neuroimaging_df_1, neuroimaging_df_2], 
                                       path_to_dataframes = '/Users/cu135/Dropbox (Partners HealthCare)/resources/datasets/BIDS_AD_DBS_FORNIX/response_topology/mediated_moderation/server_prep')
print(df_paths_dict)

# Submit to cluster


**Transfer Files From Above**

    ScpTransfer class provides a convenient way to transfer files to a remote server
    using SCP (Secure Copy).
    Will request your password for transfer if no ssh_key provided.

    Example:
        remote_base_path = "/path/on/remote/server"
        remote_hostname = "example.com"
        remote_username = "your_username"
        ssh_key = "path/to/ssh_key/"

        scp_transfer = ScpTransfer(remote_hostname, remote_username, remote_password)
        scp_transfer.transfer_files_in_dict(dict_files_to_transfer, remote_base_path)

In [ ]:
%pip install paramiko
from calvin_utils.server_utils.file_transfer_helper import ScpTransfer


#----------------------------------------------------------------
scp_transfer = ScpTransfer(hostname="erisone.partners.org",
                           username="cu135", 
                           ssh_key=None
                        )
scp_transfer.transfer_files_in_dict(dict_files=df_paths_dict, 
                                    base_remote_path='/PHShome/cu135/permutation_tests/f_test/age_by_stim_ad_dbs_redone/inputs')

**Submit Jobs**

    Initialize server
    server = LSFServer("erisone.partners.org", "cu135")

    Initialize job
    job = LSFJob(
    job_name="f_test_bm",
    user_email="choward12@bwh.harvard.edu",
    cpus=1,
    output_dir="/PHShome/cu135/terminal_outputs",
    error_dir="/PHShome/cu135/error_outputs",
    queue="big-multi",
    script_path="/PHShome/cu135/python_scripts/launch_f_test_palm.py",
    options="-o /PHShome/cu135/permutation_tests/f_test/age_by_stim_pd_dbs/results",
    Gb_requested=4,
    wait_time=0
    )

    Initialize job submitter
    job_submitter = JobSubmitter(server, job)

    Submit jobs
    job_submitter.submit_jobs(10)

This example will log into the server 'erisone.partners.org' as user 'cu135'.
 
Will submit 10 jobs named 'f_test_bm' to the queue 'big-multi'. 

Each job will run the Python script at '/PHShome/cu135/python_scripts/launch_f_test_palm.py' with the options specified.

Putputting to '/PHShome/cu135/terminal_outputs' and '/PHShome/cu135/error_outputs' for terminal and error logs respectively. 

The job requests 1 CPU and 4GB of memory.


In [ ]:
from calvin_utils.server_utils.job_submission_helper import LSFServer, LSFJob
how_many_permutations_would_you_like_to_do = 10

#----------------------------------------------------------------
# Create an instance of the LSF job.
# Assuming a resource requirement of 500GB memory and 6 cores for each job.
# And a job script with some options.

lsf_job = LSFJob(job_name="f-test_big",
                 user_email="choward12@bwh.harvard.edu",
                 output_dir="/PHShome/cu135/terminal_outputs",
                 error_dir="/PHShome/cu135/terminal_outputs",
                 queue="big",
                 n_jobs=20,
                 options="-o /PHShome/cu135/permutation_tests/f_test/age_by_stim_pd_dbs/results",
                 work_dir="/PHShome/cu135/permutation_tests",
                 cpus=6,
                 Gb_requested=498,
                 wait_time=None
                 script_path="/PHShome/cu135/python_scripts/launch_f_test_palm.py",
                 options='q'
                 )

# Create an instance of the LSF server.
lsf_server = LSFServer(server_name="erisone.parters.org", 
                       username="cu135")

In [ ]:
from calvin_utils.server_utils.job_submission_helper import JobSubmitter

# Submit to server
job_submitter = JobSubmitter(server=lsf_server, job=lsf_job)
job_submitter.submit_jobs(n_jobs=20)

# Recompose Data
This code uses a file-staging approach to large-scale computation. The resultant files have been saved to your output directory. You must now recompose them. 

In [ ]:

import os
import pandas as pd

def combine_csvs(directory, output_filename):
    """
    Combine all CSV files in a directory into a single CSV file.

    Parameters:
    - directory (str): The path to the directory containing the CSV files.
    - output_filename (str): The path to the output CSV file.
    """
    # Initialize an empty DataFrame
    combined_df = pd.DataFrame()

    # Get a list of all CSV files in the directory
    csv_files = [f for f in os.listdir(directory) if f.endswith(".csv")]

    # Loop through the CSV files and append each one to the combined DataFrame
    for csv_file in csv_files:
        df = pd.read_csv(os.path.join(directory, csv_file))
        combined_df = pd.concat([combined_df, df])

    # Save the combined DataFrame as a new CSV file
    combined_df.to_csv(output_filename, index=False)
    return output_filename


In [ ]:
recomposed_csv_path = combine_csvs("/path/to/your/directory", "combined.csv")
recomposed_csv_df = pd.read_csv(recomposed_csv_path)
recomposed_csv_df

In [ ]:
#import confidence_intervals
# run confidence intervals on the csv
import multiprocessing as mp

import numpy as np
import pandas as pd
from tqdm import tqdm
from statsmodels.formula.api import ols

def calculate_confidence_intervals(ab_paths, mediators):
    """
    Calculates the confidence intervals and p-value based on the bootstrapped samples.

    Parameters:
    - ab_paths: list of lists containing the bootstrapped ab paths for each mediator.
    - total_indirect_effects: list of bootstrapped summed ab paths.
    - mediators: list of mediator names.

    Returns:
    - DataFrame with the mean indirect effect, confidence intervals, and p-values for each mediator and the total indirect effect.
    """
    ab_path_values = np.array(ab_paths)

    # Check if there's only one mediator
    if isinstance(mediators, str):
        mediators = [mediators]

    # Calculate mean indirect effect and confidence intervals for each mediator
    mean_ab_paths = np.mean(ab_path_values, axis=0)
    lower_bounds = np.percentile(ab_path_values, 2.5, axis=0)
    upper_bounds = np.percentile(ab_path_values, 97.5, axis=0)

    # Calculate p-values for each mediator
    ab_path_p_values = [np.mean(np.sign(mean_ab_paths) * ab_path_values <= 0)]

    # Create DataFrame to store the results
    result_df = pd.DataFrame({
        'Point Estimate': mean_ab_paths,
        '2.5th Percentile': lower_bounds,
        '97.5th Percentile': upper_bounds,
        'P-value': ab_path_p_values
    }, index=mediators)

    return result_df


Get Results

In [ ]:
pool = mp.Pool(mp.cpu_count())
results = list(tqdm(pool.imap(calculate_confidence_intervals, [row for _, row in recomposed_csv_df.iterrows()]), total=recomposed_csv_df.shape[0]))
pool.close() 

# Create the results DataFrame
results_df = pd.concat(results, axis=1).transpose()

# Create Nifti Results

In [ ]:
# results_df = pd.read_csv('/Users/cu135/Dropbox (Partners HealthCare)/resources/datasets/BIDS_AD_DBS_FORNIX/response_topology/mediated_moderation/age_mediated_by_grey_matter/P-value_mediated_moderation_analysis_results.csv')

mask_threshold=0
#----------------------------------------------------------------DO NOT TOUCH
#Prepare Indices to Unmask
from calvin_utils.nifti_utils.matrix_utilities import unmask_matrix
from nimlab import datasets as nimds
from calvin_utils.nifti_utils.generate_nifti import view_and_save_nifti
from nilearn import image

#Perform Unmasking
if mask_path is not None:
    mni_mask = image.load_img(mask_path).get_fdata().flatten()
    brain_indices = np.where(mni_mask > mask_threshold)[0]
elif masking_df is not None:
    mni_mask = nimds.get_img("mni_icbm152").get_fdata().flatten()
    mask = masking_df.transpose().reset_index(drop=True).copy()
    mask['mask_index'] = mask.sum(axis=1)
    brain_indices = np.where(mask['mask_index'] != 0)[0]
else:
    mni_mask = nimds.get_img("mni_icbm152").get_fdata().flatten()
    brain_indices = np.where(mni_mask > 0)[0]

# Create a boolean mask for brain_indices
bool_mask = np.zeros_like(mni_mask, dtype=bool)
bool_mask[brain_indices] = True

for statistic in results_df.columns:
    try:
        print(statistic)
        
        # Initialize the output mask with NaN values
        output_mask = np.full_like(mni_mask, np.nan)
        
        # Reinstate the values at brain_indices
        output_mask[bool_mask] = results_df[statistic]
        
        # View and save
        view_and_save_nifti(output_mask, out_dir=out_dir, output_name=statistic)
    except:
        print(f"Couldn't convert {statistic} to nifti")

Done